Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

## Train CCNet with titanic
https://www.kaggle.com/competitions/titanic/data

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
dataset_name = "California House Price"
df = pd.read_csv(path_append + f'../data/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [3]:
from tools.preprocessing.data_frame import auto_preprocess_dataframe
target_columns = ['Survived']
drop_columns = ['PassengerId', 'Name', 'Cabin', 'Ticket']
encode_columns = ['Sex', 'Embarked', 'Pclass']
df, description = auto_preprocess_dataframe(df, target_columns, drop_columns, encode_columns)

Dropped columns: PassengerId, Name, Cabin, Ticket
Number of missing values in each column:
Age         177
Embarked      2
dtype: int64
Number of rows dropped due to missing values: 179

Column 'Embarked' has 3 unique values.
Column 'Pclass' has 3 unique values.
Column 'Sex' has 2 unique values.


,Min,Max,Mean,Std,Null Count,Scaled,Encoded
Age,-2.587574,4.459119,3.742325e-16,1.283328,0,Minmax,None
SibSp,-0.514045,4.485955,-9.979533e-18,0.930692,0,None,None
Parch,-0.432584,5.567416,-9.979533e-18,0.854181,0,None,None
Fare,-0.627088,19.907148,7.583728e-01,2.121790,0,Robust,None
Sex,0.000000,1.000000,6.362360e-01,0.481420,0,None,One-hot
Embarked_C,0.000000,1.000000,1.825843e-01,0.386597,0,None,None
Embarked_Q,0.000000,1.000000,3.932584e-02,0.194506,0,None,None
Embarked_S,0.000000,1.000000,7.780899e-01,0.415823,0,None,None
Pclass_1,0.000000,1.000000,2.584270e-01,0.438078,0,None,One_hot
Pclass_2,0.000000,1.000000,2.429775e-01,0.429183,0,None,One_hot


In [4]:
import torch
from sklearn.model_selection import train_test_split
from tools.preprocessing.template_dataset import TemplateDataset

train_df, test_df = train_test_split(df, test_size=0.3, shuffle=True, random_state=42)
# predict the next value in the sequence
train_df_x = train_df.iloc[:, :-1] # all columns except the last one
train_df_y = train_df.iloc[:, -1:] # only the last column

test_df_x = test_df.iloc[:, :-1] # all columns except the last one
test_df_y = test_df.iloc[:, -1:] # only the last column

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
trainset = TemplateDataset(train_df_x, train_df_y)
testset = TemplateDataset(test_df_x, test_df_y)

train df shape:  (498, 12)
test df shape:  (214, 12)


In [5]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

num_features = description['num_features']
num_classes = description['num_classes']
data_config = DataConfig(dataset_name = dataset_name, task_type='binary_classification', obs_shape=[num_features], label_size=num_classes, explain_size=num_features - num_classes)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'tabnet', encoder_network = 'none')
ml_params.training.num_epoch = 5
ml_params.training.batch_size = 32
ml_params.model.ccnet_config.num_layers = 4

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=False, use_wandb=False) 

Trainer Name: causal_trainer


ModelParameters Parameters:


,ccnet_config,ccnet_network,encoder_config,encoder_network
0,See details below,tabnet,None,none



Detailed ccnet_config Configuration:


,ccnet_config_model_name,ccnet_config_num_layers,ccnet_config_d_model,ccnet_config_dropout,ccnet_config_obs_shape
0,tabnet,4,256,0.05,[11]


TrainingParameters Parameters:


...

,clip_grad_range,decay_rate_100k,learning_rate,max_grad_norm,scheduler_type
0,None,0.05,0.0002,1.0,exponential


AlgorithmParameters Parameters:


,enable_diffusion,error_function,reset_pretrained
0,False,mse,False


DataConfig Parameters:


,dataset_name,task_type,obs_shape,label_size,explain_size,explain_layer,state_size,show_image_indices
0,california-house-price,binary_classification,[11],1,10,tanh,None,None


## Proving independence of X and E

In [6]:
import numpy as np
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=64, shuffle=False, drop_last=False)

all_labels = []

for _, labels in test_loader:
    labels = labels.numpy().flatten()  
    all_labels.extend(labels)

all_labels = np.array(all_labels)

unique_labels, counts = np.unique(all_labels, return_counts=True)

label_ratios = counts / counts.sum()

for label, ratio in zip(unique_labels, label_ratios):
    print(f'Label: {label}, Ratio: {ratio:.4f}')

Label: 0.0, Ratio: 0.5701
Label: 1.0, Ratio: 0.4299


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, 256)
        self.layer2 = nn.Linear(256, 256)
        self.layer3 = nn.Linear(256, 1)  
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        return self.sigmoid(x)

In [8]:
import tqdm

def train_classifier(model, dataset, num_epoch=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Ensure reproducibility by resetting the random seed
    # Create DataLoader for batch processing
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
    # Training loop


    metrics = {
        'loss': [],
        'accuracy': [], 
        'f1_score': []
    }
    
    len_trainset = len(dataset)
    for epoch in range(num_epoch):  # Train for 2 epochs as an example
        epoch_loss = []
        for i, (data, label) in enumerate(trainloader):
            data = torch.tensor(data).to(device).float()
            label = torch.tensor(label).to(device).float()
            # Perform forward pass
            output = model(data)
            # Compute loss
            loss = torch.nn.functional.binary_cross_entropy(output, label)
            # Backward pass to compute gradients
            loss.backward()
            # Update weights
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()
            epoch_loss.append(loss.item())
            
        avg_loss = sum(epoch_loss) / len(epoch_loss)
        print(f'Epoch {epoch+1}/{num_epoch}, Loss: {avg_loss:.4f}')
        metrics['loss'].append(avg_loss)
    return metrics

In [9]:
def test_model(model, _testset):
    val_loader = DataLoader(_testset, batch_size=32, shuffle=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.float().to(device), target.float().to(device)
            output = model(data)
            predicted = (output > 0.5).float()
            y_pred.extend(predicted.view(-1).tolist())
            y_true.extend(target.view(-1).tolist())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return acc, f1

### Before experimenting with the model, train SimpleNN model with raw data

In [10]:
model = SimpleNN(num_features).to(device)
train_metrics = train_classifier(model, trainset)

Epoch 1/10, Loss: 0.6441
Epoch 2/10, Loss: 0.5284
Epoch 3/10, Loss: 0.5350
Epoch 4/10, Loss: 0.4812
Epoch 5/10, Loss: 0.4799
Epoch 6/10, Loss: 0.4051
Epoch 7/10, Loss: 0.4231
Epoch 8/10, Loss: 0.3968
Epoch 9/10, Loss: 0.3744
Epoch 10/10, Loss: 0.3968


### load test data and ccnet from trainer_hub

In [11]:
# test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=32, shuffle=False, drop_last=False)

### 1-1 Infer Y from X created using original Explanation

In [12]:
for i in range(10):
    if i > 0:
        trainer_hub.train(trainset)
        
    causal_model = trainer_hub.ccnet
        
    # for data, labels in test_loader:
    # how to  get x and y from testset not test_loader
    test_data, test_label = testset.X, testset.y
    test_data = torch.tensor(test_data).float()
    test_label = torch.tensor(test_label).float()
    onbe_hot_labels = torch.nn.functional.one_hot(test_label.to(torch.long), num_classes=2).float().squeeze(-2)

    explanation = causal_model.explain(test_data.to(device))
    synthetic_data0 = causal_model.produce(onbe_hot_labels.to(device), explanation)
    synthetic_data1 = causal_model.produce(onbe_hot_labels.to(device), torch.cat((explanation[1:], explanation[:1]), dim=0))
    synthetic_data2 = causal_model.produce(onbe_hot_labels.to(device), torch.cat((explanation[5:], explanation[:5]), dim=0))

    synthetic_dataset0 = TemplateDataset(test_data.detach(), test_label.detach())
    synthetic_dataset1 = TemplateDataset(synthetic_data0.detach().cpu(), test_label.detach())
    synthetic_dataset2 = TemplateDataset(synthetic_data1.detach().cpu(), test_label.detach())
    synthetic_dataset3 = TemplateDataset(synthetic_data2.detach().cpu(), test_label.detach())

    test_accuracy0, test_f10 = test_model(model, synthetic_dataset0)
    test_accuracy1, test_f11 = test_model(model, synthetic_dataset1)
    test_accuracy2, test_f12 = test_model(model, synthetic_dataset2)
    test_accuracy3, test_f13 = test_model(model, synthetic_dataset3)

    print(f"Test: Accuracy: {test_accuracy0:.4f}, Test F1 Score: {test_f10:.4f}")
    print(f"No shifted: Accuracy: {test_accuracy1:.4f}, No shifted F1 Score: {test_f11:.4f}")
    print(f"First Shifted: Accuracy: {test_accuracy2:.4f}, First Shifted F1 Score: {test_f12:.4f}")
    print(f"Second Shifted: Accuracy: {test_accuracy3:.4f}, Second Shifted F1 Score: {test_f13:.4f}")


Test: Accuracy: 0.8364, Test F1 Score: 0.8087
No shifted: Accuracy: 0.4393, No shifted F1 Score: 0.6104
First Shifted: Accuracy: 0.4486, First Shifted F1 Score: 0.6194
Second Shifted: Accuracy: 0.4579, Second Shifted F1 Score: 0.6282


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Iterations:   0%|          | 0/15 [00:00<?, ?it/s]

Iterations:   0%|          | 0/15 [00:00<?, ?it/s]

Iterations:   0%|          | 0/15 [00:00<?, ?it/s]

...

Test: Accuracy: 0.8364, Test F1 Score: 0.8087
No shifted: Accuracy: 0.8925, No shifted F1 Score: 0.8743
First Shifted: Accuracy: 0.7804, First Shifted F1 Score: 0.7345
Second Shifted: Accuracy: 0.7897, Second Shifted F1 Score: 0.7305


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Iterations:   0%|          | 0/15 [00:00<?, ?it/s]

Iterations:   0%|          | 0/15 [00:00<?, ?it/s]

Iterations:   0%|          | 0/15 [00:00<?, ?it/s]

### Plotting the results